In [1]:
import numpy as np
import pytpc
from pytpc.fitting.mcopt_wrapper import Minimizer
from pytpc.constants import *
import pytpc.simulation
import pytpc.tpcplot
import pytpc.vmedata
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import h5py
from scipy.interpolate import UnivariateSpline, interp1d
from scipy.optimize import differential_evolution, minimize, basinhopping, leastsq
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import os
#import cmaps
import yaml
import scipy
from copy import copy, deepcopy
from pytpc.utilities import find_exclusion_region, read_lookup_table
import math
from effsim.paramgen import uniform_param_generator
from effsim.effsim import EventSimulator
from math import pi

/home/chen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open('/home/chen/Real Data/config_e15503a_runs_92-104.yml', 'r') as f:
    config = yaml.load(f)

mcfitter = pytpc.fitting.MCFitter(config)
num_iters = config['num_iters']
num_pts = config['num_pts']
red_factor = config['red_factor']

In [3]:
#Good events for run_0102 are: 689, 765, 896, 305, 504, 575, 456, 299, 399, 156, 231, 7832, 1568, 1354, 789, 485, 2456, 357, 4568, 1448, 6587, 3578, 745, 430
full = h5py.File('/home/chen/Real Data/clean_run_0102.h5', 'r')
evt_ID = 299
dataset_name = '/clean'
evt_full = full[dataset_name]
xyzs_h5 = evt_full[str(evt_ID)]

xyzs = np.array(xyzs_h5)

#delete the points which are farther away from the center of the spiral
del_list = []

for i in range(len(xyzs)):
    if (xyzs[i,6]) > 150.0:
        del_list.append(i)
cut_xyzs = np.delete(xyzs,del_list,axis=0)

In [4]:
xy = xyzs[:, 0:2]
xy_C = np.ascontiguousarray(xy, dtype=np.double)
cx, cy = pytpc.cleaning.hough_circle(xy_C)

cut_xy = cut_xyzs[:, 0:2]
cut_xy_C = np.ascontiguousarray(cut_xy, dtype=np.double)
cut_cx, cut_cy = pytpc.cleaning.hough_circle(cut_xy_C)

In [5]:
uvw, (cu, cv) = mcfitter.preprocess(xyzs[:,0:5], center=(cx, cy), rotate_pads=False)
uvw_values = uvw.values

cut_uvw, (cut_cu, cut_cv) = mcfitter.preprocess(cut_xyzs[:,0:5], center=(cut_cx, cut_cy), rotate_pads=False) # get calibrated set of data
cut_uvw_values = cut_uvw.values

In [6]:
uvw_sorted = uvw.sort_values(by='w', ascending=True)
prefit_data = uvw_sorted.iloc[-len(uvw_sorted) // 4:].copy()
prefit_res = mcfitter.linear_prefit(prefit_data, cu, cv)
ctr0 = mcfitter.guess_parameters(prefit_res)
exp_pos = uvw_sorted[['u', 'v', 'w']].values.copy() / 1000

cut_uvw_sorted = cut_uvw.sort_values(by='w', ascending=True)
cut_prefit_data = cut_uvw_sorted.iloc[-len(cut_uvw_sorted) // 4:].copy()
cut_prefit_res = mcfitter.linear_prefit(cut_prefit_data, cut_cu, cut_cv)
cut_ctr0 = mcfitter.guess_parameters(cut_prefit_res)
cut_exp_pos = cut_uvw_sorted[['u', 'v', 'w']].values.copy() / 1000

In [7]:
exp_hits = np.zeros(10240)
for a, p in uvw[['a', 'pad']].values:
    exp_hits[int(p)] = a
    
cut_exp_hits = np.zeros(10240)
for a, p in cut_uvw[['a', 'pad']].values:
    cut_exp_hits[int(p)] = a

In [8]:
minimizer = Minimizer(mcfitter.tracker, mcfitter.evtgen, num_iters, num_pts, red_factor)

In [9]:
def f(y):
    ctr = np.zeros([1,6])
    ctr[0] = y
    chi_result = minimizer.run_tracks(ctr, exp_pos, exp_hits)
    return sum(chi_result[0])
eps = np.sqrt(np.finfo(np.float).eps)
def jacobian(x):
    return scipy.optimize.approx_fprime(x,f,eps)

In [10]:
def cut_f(y):
    ctr = np.zeros([1,6])
    ctr[0] = y
    chi_result = minimizer.run_tracks(ctr, cut_exp_pos, cut_exp_hits)
    return sum(chi_result[0])
eps = np.sqrt(np.finfo(np.float).eps)
def cut_jacobian(x):
    return scipy.optimize.approx_fprime(x,cut_f,eps)

In [11]:
i = 1
def callbackF(x):
    global i
    print('{0:4d}{1: 3.6f}{2: 3.6f}{3: 3.6f}{4: 3.6f}{5: 3.6f}{6: 3.6f}{7: 3.6f}'.format(i,x[0],x[1],x[2],x[3],x[4],x[5],f(x)))
    i += 1   

In [12]:
j = 1    
def cut_callbackF(x):
    global j
    print('{0:4d}{1: 3.6f}{2: 3.6f}{3: 3.6f}{4: 3.6f}{5: 3.6f}{6: 3.6f}{7: 3.6f}'.format(j,x[0],x[1],x[2],x[3],x[4],x[5],cut_f(x)))
    j += 1  

In [ ]:
scipy.optimize.minimize(f, ctr0, jac=jacobian, method="CG", callback=callbackF)

In [ ]:
scipy.optimize.minimize(cut_f, cut_ctr0, jac=cut_jacobian, method="CG", callback=cut_callbackF)

In [ ]:
nm = scipy.optimize.minimize(f, ctr0, method="Nelder-Mead", callback=callbackF)    

In [ ]:
scipy.optimize.minimize(cut_f, cut_ctr0, method="Nelder-Mead", callback=cut_callbackF)

In [ ]:
powell = scipy.optimize.minimize(f, ctr0, method="Powell", callback=callbackF)    

In [ ]:
scipy.optimize.minimize(cut_f, cut_ctr0, method="Powell", callback=cut_callbackF)    

In [ ]:
scipy.optimize.minimize(f, ctr0, method="BFGS", callback=callbackF)    

In [ ]:
scipy.optimize.minimize(cut_f, cut_ctr0, method="BFGS", callback=cut_callbackF)    

In [ ]:
scipy.optimize.minimize(f, ctr0, method="Newton-CG", jac=jacobian, callback=callbackF)    

In [ ]:
scipy.optimize.minimize(cut_f, cut_ctr0, method="Newton-CG", jac=cut_jacobian, callback=cut_callbackF)    

In [ ]:
l_bfgs_b = scipy.optimize.minimize(f, ctr0, method="L-BFGS-B", jac=jacobian, callback=callbackF)  

In [ ]:
scipy.optimize.minimize(cut_f, cut_ctr0, method="L-BFGS-B", jac=cut_jacobian, callback=cut_callbackF)  

In [ ]:
tnc = scipy.optimize.minimize(f, ctr0, method="TNC", jac=jacobian, callback=callbackF)  

In [ ]:
scipy.optimize.minimize(cut_f, cut_ctr0, method="TNC", jac=cut_jacobian, callback=cut_callbackF)  

In [ ]:
scipy.optimize.minimize(f, ctr0, method="COBYLA")  

In [ ]:
scipy.optimize.minimize(cut_f, cut_ctr0, method="COBYLA")  

In [ ]:
scipy.optimize.minimize(f, ctr0, method="SLSQP")  

In [ ]:
scipy.optimize.minimize(cut_f, cut_ctr0, method="SLSQP")  

In [13]:
scipy.optimize.anneal(f,ctr0)

AttributeError: module 'scipy.optimize' has no attribute 'anneal'

In [ ]:
def hessian (x, the_func, epsilon=1e-8):
    """Numerical approximation to the Hessian
    Parameters
    ------------
    x: array-like
        The evaluation point
    the_func: function
        The function. We assume that the function returns the function
value and
        the associated gradient as the second return element
    epsilon: float
        The size of the step
    """

    N = x.size
    h = np.zeros((N,N))
    df_0 = the_func(x)[1]
    for i in xrange(N):
        xx0 = 1.*x[i]
        x[i] = xx0 + epsilon
        df_1 = the_func(x)[1]
        h[i,:] = (df_1 - df_0)/epsilon
        x[i] = xx0
    return h

def hess(x):
    return hessian(x,f,eps)

scipy.optimize.minimize(f, ctr0, method="trust-exact", jac=jacobian, hess=hess)